## 4) Dates partie 2 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://bit.ly/felonies-dataset")

In [4]:
df.head()

,Date of Arrest,Age,Convicted
0,2014-07-16 14:59:18,77,Yes
1,2002-12-19 22:17:59,28,Yes
2,1994-06-28 09:31:40,21,No
3,1985-05-14 09:52:17,45,Yes
4,1985-05-25 14:06:28,39,Yes


In [5]:
df.shape

(1000, 3)

In [6]:
df.dtypes

Date of Arrest    object
Age                int64
Convicted         object
dtype: object

### La méthode 'Map' de Pandas

In [7]:
df['DoA'] = pd.to_datetime(df['Date of Arrest'])

In [8]:
df.dtypes

Date of Arrest            object
Age                        int64
Convicted                 object
DoA               datetime64[ns]
dtype: object

In [14]:
df.drop('Date of Arrest', axis='columns', inplace=True)

In [15]:
df

,Age,Convicted,DoA
0,77,Yes,2014-07-16 14:59:18
1,28,Yes,2002-12-19 22:17:59
2,21,No,1994-06-28 09:31:40
3,45,Yes,1985-05-14 09:52:17
4,39,Yes,1985-05-25 14:06:28
...,...,...,...
995,72,Yes,1992-02-04 14:24:25
996,68,Yes,1996-04-29 18:03:23
997,87,Yes,1980-07-15 21:59:27
998,27,No,1999-07-05 17:16:12


In [10]:
df.DoA.dt.dayofweek

0      2
1      3
2      1
3      1
4      5
      ..
995    1
996    0
997    1
998    0
999    6
Name: DoA, Length: 1000, dtype: int64

In [11]:
dayofweek2weektime = {0:'DEBUT', 1:'DEBUT', 
                      2:'MILIEU', 3:'MILIEU', 4:'MILIEU', 
                      5:'WEEKEND', 6:'WEEKEND'}

In [13]:
df.DoA.dt.dayofweek.map(dow2weektime)

NameError: name 'dow2weektime' is not defined

### Une liste /un dictionnaire

In [126]:
id2str = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY','SUNDAY',]

In [17]:
#id2str.index() #Parcours la liste valeur par valeur --> long

TypeError: index expected at least 1 argument, got 0

In [32]:
str2id = {value:idx for idx, value in enumerate(id2str)} # à connaitre
id2str = {value:idx for idx, value in enumerate(id2str)} # à connaitre

In [19]:
str2id

{'MONDAY': 0,
 'TUESDAY': 1,
 'WEDNESDAY': 2,
 'THURSDAY': 3,
 'FRIDAY': 4,
 'SATURDAY': 5,
 'SUNDAY': 6}

In [20]:
jour = str2id

In [21]:
jour['MONDAY']

0

In [33]:
#str2id[id2str[0]]

KeyError: 0

In [34]:
df['DoA'].dt.dayofweek.map(id2str)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
995   NaN
996   NaN
997   NaN
998   NaN
999   NaN
Name: DoA, Length: 1000, dtype: float64

In [35]:
periodeSemaine = {'MONDAY': 'DEBUT',
                 'TUESDAY': 'DEBUT',
                 'WEDNESDAY': 'MILIEU',
                 'THURSDAY': 'MILIEU',
                 'FRIDAY': 'MILIEU',
                 'SATURDAY': 'WEEKEND',
                 'SUNDAY': 'WEEKEND'}

In [41]:
df['DoA'].dt.dayofweek.map(id2str).map(periodeSemaine) # Double map

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
995    NaN
996    NaN
997    NaN
998    NaN
999    NaN
Name: DoA, Length: 1000, dtype: object

In [40]:
df['Période Semaine'] = df['DoA'].dt.dayofweek.map(id2str).map(periodeSemaine) #à completer

In [42]:
df.dtypes

Age                         int64
Convicted                  object
DoA                datetime64[ns]
Péridoe Semaine            object
Période Semaine            object
dtype: object

### Convertir la colonne en type categorie [ORDINAL]  --> Exploration

In [43]:
ordre = ['DEBUT', 'MILIEU', 'WEEKEND']

In [48]:
df['Période Semaine'] = (df['Période Semaine'].astype('category').
                         cat.reorder_categories(ordre).
                         cat.asordered()) #erreur à corriger

ValueError: items in new_categories are not the same as in old categories

### Convertir la colonne " category" en "int" [ORDINAL] --> ML

In [51]:
df['Période Semaine'] = df['Période Semaine'].cat.codes

AttributeError: Can only use .cat accessor with a 'category' dtype

### Gérer les cycles

In [53]:
import pandas as pd
fp = pd.read_csv("https://bit.ly/felonies-dataset")
df = pd.read_csv(fp)

df.rename(colums=['Date of Arrest' : 'DoA'], inplace=True)

df.dtypes

SyntaxError: invalid syntax (<ipython-input-53-34a97c52552e>, line 5)

In [54]:
df['DoA'] = pd.to_datetime(df['DoA'])
df.dtypes

Age                         int64
Convicted                  object
DoA                datetime64[ns]
Péridoe Semaine            object
Période Semaine            object
dtype: object

In [62]:
df['DoA'].dt.day.max()

31

In [63]:
df['DoA'].dt.month.max()

12

In [58]:
colonne_temporaire = df['DoA'].dt.month * 100 + df['DoA'].dt.day

In [59]:
colonne_temporaire

0       716
1      1219
2       628
3       514
4       525
       ... 
995     204
996     429
997     715
998     705
999     317
Name: DoA, Length: 1000, dtype: int64

In [66]:
AVANT_JANVIER=0
AVANT_PRINTEMPS=320
AVANT_ETE=620
AVANT_AUTOMNE=920
AVANT_HIVER=1220
FIN_ANNEE=1232

bins=[AVANT_JANVIER,
      AVANT_PRINTEMPS,
      AVANT_ETE,
      AVANT_AUTOMNE,
      AVANT_HIVER,
      FIN_ANNEE]

In [70]:
saisons = pd.cut(colonne_temporaire,
                 bins=bins,
                 labels= ['HIVER', 'PRINTEMPS', 'ETE', 'AUTOMNE', 'HIVER2'])

In [72]:
saisons.head()

0          ETE
1      AUTOMNE
2          ETE
3    PRINTEMPS
4    PRINTEMPS
Name: DoA, dtype: category
Categories (5, object): ['HIVER' < 'PRINTEMPS' < 'ETE' < 'AUTOMNE' < 'HIVER2']

In [85]:
saisons = saisons.str.replace('2', '') #objet
saisons.unique()

array(['ETE', 'AUTOMNE', 'PRINTEMPS', 'HIVER'], dtype=object)

In [84]:
saisons.head()

0          ETE
1      AUTOMNE
2          ETE
3    PRINTEMPS
4    PRINTEMPS
Name: DoA, dtype: object

In [81]:
saison = (saisons.astype('category').
          cat.as_ordered().
          cat.reorder_categories(['AUTOMNE', 'HIVER', 'PRINTEMPS', 'ETE']))

In [82]:
saison

0            ETE
1        AUTOMNE
2            ETE
3      PRINTEMPS
4      PRINTEMPS
         ...    
995        HIVER
996    PRINTEMPS
997          ETE
998          ETE
999        HIVER
Name: DoA, Length: 1000, dtype: category
Categories (4, object): ['AUTOMNE' < 'HIVER' < 'PRINTEMPS' < 'ETE']

### --------------------------------------------------------------------------------------------------------------------------------------------------------

### Exercice avec Heures 

Sujet:
22h12 --> 3H24 / NUIT; 
3h25 --> 16h12 / MATIN;
16h13 --> 22h11 / AM;

In [174]:
import pandas as pd
fp = 'https://bit.ly/felonies-dataset'
df = pd.read_csv(fp)

df.rename(columns = {'Date of Arrest': 'DoA'}, inplace = True)

df.dtypes

DoA          object
Age           int64
Convicted    object
dtype: object

In [175]:
df['DoA'] = pd.to_datetime(df['DoA'])
df.dtypes

DoA          datetime64[ns]
Age                   int64
Convicted            object
dtype: object

In [176]:
colonne_minutes = df['DoA'].dt.hour * 100 + df['DoA'].dt.minute

In [177]:
colonne_minutes

0      1459
1      2217
2       931
3       952
4      1406
       ... 
995    1424
996    1803
997    2159
998    1716
999    1829
Name: DoA, Length: 1000, dtype: int64

In [178]:
DEBUT_JOUR=0
AVANT_MATIN=324
AVANT_AM=1612
AVANT_NUIT=2211
FIN_JOUR=2359

bins=[DEBUT_JOUR,
      AVANT_MATIN,
      AVANT_AM,
      AVANT_NUIT,
      FIN_JOUR]

In [179]:
periodeJournee = pd.cut(colonne_minutes,
                 bins=bins,
                 labels=['NUIT', 'MATIN', 'AM', 'NUIT2'])

In [180]:
colonne_heures = df['DoA'].dt.hour * 100

In [181]:
periodeJour = periodeJournee.str.replace('2', '') #objet
periodeJournee.unique()

['MATIN', 'NUIT2', 'NUIT', 'AM']
Categories (4, object): ['NUIT' < 'MATIN' < 'AM' < 'NUIT2']

In [185]:
periodeJour.head(10)

0    MATIN
1     NUIT
2    MATIN
3    MATIN
4    MATIN
5    MATIN
6     NUIT
7    MATIN
8    MATIN
9     NUIT
Name: DoA, dtype: object

### -----------------------------------------------------------------------------------------------------------------------------------------------------


Sauvegarder un objet Python

import pickle # sauvegarder un objet python

chemin = ('objet.pkl')

#En ecriture
with open(chemin, 'wb') as f:
    pickle.dump(id2str, f) #jeter(dump) / cornichons(OBJET/id2str) / bocal(f)

#En lecture
with open(chemin, 'rb') as f:
    objet = pickle.load(f)